In [1]:
import pandas as pd

In [6]:
from module import filter_data
from astroquery.utils.tap.core import TapPlus # library to query astronomical databases, TAP services

tap = TapPlus(url="http://tap.roe.ac.uk/ssa")

adql = """
SELECT TOP 1000
    z.*,
    p.*
FROM BestDR9.ZooSpec AS z 
JOIN BestDR7.PhotoObj AS p
  ON p.objid = z.dr7objid
"""
#selects all the columns of both tables and joins them on the objid

# Run synchronously, the code will wait until the job is finished, only after that it will continue
# Asynchronous the program sends the adql query and continue running the code
job = tap.launch_job(adql)
results = job.get_results()
df = results.to_pandas() # convert to pandas DataFrame


df_filtered = filter_data(df, 0.9)
df_filtered = df_filtered.reset_index()

ModuleNotFoundError: No module named 'PIL'

In [ ]:
from module import get_image
import os
from PIL import Image
from concurrent.futures import ThreadPoolExecutor, as_completed

photodir = "galaxy_images2"
if not os.path.exists(photodir):
    os.mkdir(photodir)


def get_galaxy(gal):
    pack = int(gal / 1000)
    packdir = os.path.join(photodir, f"pack{pack}")
    if not os.path.exists(packdir):
        os.mkdir(packdir)
    result = get_image(
        df_filtered.loc[gal, "ra"],
        df_filtered.loc[gal, "dec"],
        df_filtered.loc[gal, "petroR90_r"],
        64,
        64,
    )
    im = Image.fromarray(result)
    im = im.save(os.path.join(packdir, f"g{gal}.jpg"))
    return f"Saved g{gal}"


MAX_WORKERS = 16  # You can adjust based on CPU/network
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(get_galaxy, gal) for gal in range(min(len(df_filtered), 70000))]

    for future in as_completed(futures):
        print(future.result())

KeyboardInterrupt: 